<font size="5">**Summary**</font>

<font size="5">**Problem description**</font>

The U.S. has almost 500 students for every guidance counselor. Underserved youth lack the network to find their career role models, making CareerVillage.org the only option for millions of young people in America and around the globe with nowhere else to turn.

To date, 25,000 volunteers have created profiles and opted in to receive emails when a career question is a good fit for them. This is where your skills come in. To help students get the advice they need, the team at CareerVillage.org needs to be able to send the right questions to the right volunteers. The notifications sent to volunteers seem to have the greatest impact on how many questions are answered,

So the objective of this Kernel is to develop a method to recommend relevant questions to the professionals who are most likely to answer them.

<font size="5">Proposed solution</font>

From the fact that the professionals will supposed to answer similar questions they previously answered , so getting a similar questions to the asked question will help a lot ,
so the supposed recommender consists of two parts :
1.  In the first module the recommendations are based on the similarities between the question asked by a student and the pre-answered questions by the professionals in the system , where the recommended professionals from this part are the professionals who anweser similar questions and ranked by their activity status (the most recent active one has higher score than the less active one) .
2. In this module the recommendations are based on the idea that professionals similar to another professional can be used to predict how much this professional will answer a particular question even if he didn’t answer a similar question before.



# Kernel content
   ## 1. [EDA](#eda)
   ## 2. [Content based filtering](#cb)
   ## 3. [Collaborative filtering](#cf)
  ## 4. [Testing whole model](#tst)
  ## 5. [How to use the model](#use)


The recommendation system is a hybrid of content based filtering and collaborative filtering , so the output of content based which named by **ToCollaborative** is passed from content based filtering to collaborative filtering and the final output is sorted_output

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')
import operator

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.cluster import KMeans

import re
import string 
from collections import Counter
from nltk.corpus import stopwords
stop = stopwords.words('english')

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import warnings
warnings.simplefilter('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

## Read All csv's

In [ ]:
emails = pd.read_csv('../input/emails.csv')
questions = pd.read_csv('../input/questions.csv')
professionals = pd.read_csv('../input/professionals.csv')
comments = pd.read_csv('../input/comments.csv')
tag_users = pd.read_csv('../input/tag_users.csv')
group_memberships = pd.read_csv('../input/group_memberships.csv')
tags = pd.read_csv('../input/tags.csv')
students = pd.read_csv('../input/students.csv')
groups = pd.read_csv('../input/groups.csv')
tag_questions = pd.read_csv('../input/tag_questions.csv')
matches = pd.read_csv('../input/matches.csv')
answers = pd.read_csv('../input/answers.csv')
school_memberships = pd.read_csv('../input/school_memberships.csv')
question_score = pd.read_csv('../input/question_scores.csv')
Answer_score = pd.read_csv('../input/answer_scores.csv')

## EDA<a id='eda'></a> 
### 1. [Exploring students](#stud)
### 2. [Exploring questions](#quest)
### 3. [Exploring students with questions](#st_qt)
### 4. [Exploring Professionals](#prof)
### 5. [Exploring comments](#comm)
### 6. [Exploring professionals with comments](#prof_comm)
### 7. [Exploring Answers](#ans)
### 8. [Exploring questions with answers](#qt_ans)
### 9. [Exploring tags](#tag)
### 10.[Exploring tags with tag_users](#tag_tgU)
### 11.[Exploring Emails](#em)
### 12.[Exploring matches](#mat)

### Let's explore students<a id='stud'></a>

In [ ]:
print(students.shape)
students.head(5)

There are NAN values ... Let's clean it and continue exploring :D

In [ ]:
students=students.dropna(subset = ['students_id','students_location', 'students_date_joined'])
print(students.shape)
students.head(5)

2033 rows are deleted ... let's continue exploring :D

** What is the most region the students comes from ?!**

In [ ]:
students['students_location'].nunique()

In [ ]:
top10_regions_stud = students['students_location'].value_counts().head(10)
ax = top10_regions_stud.plot.bar(x=top10_regions_stud.index, y=top10_regions_stud.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('students count by location')
#ax.set_facecolor("black")

New York is the largest region the students come from ...

Let's see distribution of number of students over years

In [ ]:
date=students['students_date_joined'].str.split('-').values
years=[]
for i in range(0,len(date)):
    years.append(date[i][0])

stud_join_years = pd.Series(years)
stud_join_years_counts=stud_join_years.value_counts()
stud_join_years_counts.sort_index(inplace=True)
ax = stud_join_years_counts.plot.bar(x=stud_join_years_counts.index, y=stud_join_years_counts.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('number of student over years')

Seems that the community became most popular in 2016 

### Let's explore questions<a id='quest'></a>

In [ ]:
print(questions.shape)
questions.head(5)

In [ ]:
questions.isnull().any()

Great there is no NAN values .. let's continue exploring

let's see if the question body need some cleaning

In [ ]:
print(questions.iloc[0]['questions_body'])
print(questions.iloc[30]['questions_body'])
print(questions.iloc[120]['questions_body'])
print(questions.iloc[300]['questions_body'])

great it doesn't need cleaning .. it is in good english format

What are the most question's titles about?

In [ ]:
all_quest_titles=questions['questions_title'].str.cat(sep=' ')

wordcloud = WordCloud(width=1500, height=1500).generate(all_quest_titles)

plt.figure(figsize=(20, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

most of the questions' title about College , Career and jobs

Let's see distribution of questions over years

In [ ]:
question_date=questions['questions_date_added'].str.split('-').values
years_questions=[]
#months_questions=[]
for i in range(0,len(question_date)):
    years_questions.append(question_date[i][0])
    #months_questions.append(question_date[i][1])

    
questions_date_added = pd.Series(years_questions)
questions_date_added_counts=questions_date_added.value_counts()

questions_date_added_counts.sort_index(inplace=True)
ax = questions_date_added_counts.plot.bar(x=questions_date_added_counts.index, y=questions_date_added_counts.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('number of questions over years')


2016 is the year with largest number of questions ... seems that the community becomes more famous from year 2016

### Let's explore students with questions<a id='st_qt'></a>

First let's merge students and questions tables

In [ ]:
students_questions = pd.merge(students, questions, left_on='students_id',right_on='questions_author_id', how='inner')
students_questions.head()

Who are the most active students ?

In [ ]:
no_questions_Bystudent=students_questions.groupby('students_id').size()
top10_activeStudents=no_questions_Bystudent.sort_values(ascending=False).head(10)
ax = top10_activeStudents.plot.bar(x=top10_activeStudents.index, y=top10_activeStudents.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('number of questions asked by students')

### Let's explore professionals<a id='prof'></a>

In [ ]:
print(professionals.shape)
professionals.head(5)



In [ ]:
professionals.isnull().any()

In [ ]:
professionals=professionals.dropna(subset = ['professionals_location', 'professionals_industry', 'professionals_headline'])
print(professionals.shape)
professionals.head(5)

5248 rows are deleted ... let's continue exploration

In [ ]:
professionals['professionals_industry'].value_counts()

In [ ]:
top10_prof_industries = professionals['professionals_industry'].value_counts().head(10)
ax = top10_prof_industries.plot.bar(x=top10_prof_industries.index, y=top10_prof_industries.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('count professionals by Industry')

Information Technology and Services has the largest number of professionals

What are the most regions the professionals come from ?

In [ ]:
professionals['professionals_location'].nunique()

In [ ]:
top10_prof_regions = professionals['professionals_location'].value_counts().head(10)
ax = top10_prof_regions.plot.bar(x=top10_prof_regions.index, y=top10_prof_regions.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('count professionals by location')

The largest number of heros from New York

What are the most frequent headlines of professionals?

In [ ]:
professionals['professionals_headline'].value_counts().sort_values(ascending=False).head(10)


There are 176 professionals without headlines ... Let's remove these rows and continue exploring

In [ ]:
professionals=professionals[professionals['professionals_headline'] != '--']
professionals.shape

176 rows are removed ... Let's continue exploration

In [ ]:
top10_profHeadlines = professionals['professionals_headline'].value_counts().sort_values(ascending=False).head(10)
ax = top10_profHeadlines.plot.bar(x=top10_profHeadlines.index, y=top10_profHeadlines.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('count professionals by headlines')

Assurance Associate at PwC and Software Engineer are the most two frequent headlines of professionals 

### Let's explore comments<a id='comm'></a> 

In [ ]:
comments.head(5)

Let's see the hottest questions with the largest number of comments ...

In [ ]:
top10_number_comments =comments['comments_parent_content_id'].value_counts().head(10)
ax = top10_number_comments.plot.bar(x=top10_number_comments.index, y=top10_number_comments.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('count comments number for questions')
ax.set_xlabel("question id")

### Exploring professionals with comments<a id='prof_comm'></a>

Let's explore comments with professionals and see the most interactive professionals through comments ..

In [ ]:
comments_profs = pd.merge(comments, professionals, left_on='comments_author_id',right_on='professionals_id',how='inner')
comments_profs                          

In [ ]:
top10_profID_inComments = comments_profs.groupby('professionals_id').size().sort_values(ascending=False).head(10)
ax = top10_profID_inComments.plot.bar(x=top10_profID_inComments.index, y=top10_profID_inComments.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('Top10 professionals make comments')

In [ ]:
top10_prof_inComments_df = professionals.loc[professionals['professionals_id'].isin(top10_profID_inComments.index)]
top10_prof_inComments_df['professionals_industry']

The top active professional in comments from Design industry

Is there a relations between number of professionals in an industry and number of comments in this industry ?

In [ ]:
top10_industry_comments=comments_profs.groupby('professionals_industry').size().sort_values(ascending=False).head(10)
#ax = top10_industry_comments.plot.bar(x=top10_industry_comments.index, y=top10_industry_comments.values)
#print(top10_industry_comments)

prof_in_top10Industry_df = professionals.loc[professionals['professionals_industry'].isin(top10_industry_comments.index)]
top10_prof_comments=prof_in_top10Industry_df.groupby('professionals_industry').size()                                         

#print(top10_prof_comments)

commentsVsProf_inIndustry_df=pd.concat([top10_industry_comments.rename('no.comments'), top10_prof_comments.rename('no.profs')], axis=1 )
ax = commentsVsProf_inIndustry_df.plot.bar()
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('no.comments , no.profs VS Industry')
ax.set_xlabel("Industry name")

unexpectedly there is no relation between number of comments and professionals in the same industry

### Let's explore Answers<a id='ans'></a>

In [ ]:
print(answers.shape)
answers.head(5)

### Let's explore Questions with answers<a id='qt_ans'></a>

In [ ]:
questions.head(5)

In [ ]:
answers.head(5)

Let's combine Questions and Answers

In [ ]:
questions_Answers = pd.merge(questions, answers, left_on='questions_id',right_on='answers_question_id', how='inner')
questions_Answers.head(5)

are all the questions take the same time to be answered ?

In [ ]:
questions_Answers['questions_date_added'] = pd.to_datetime(questions_Answers['questions_date_added'])
questions_Answers['answers_date_added'] = pd.to_datetime(questions_Answers['answers_date_added'])
questions_Answers['respond_time'] = (questions_Answers['answers_date_added']-questions_Answers['questions_date_added']).dt.days  
#questions_Answers['respond_time'].head(10)
ax=questions_Answers['respond_time'].plot()
ax.set_xlim([0,2000])
ax.set_title('questions respond time')
ax.set_xlabel("respond time in days")

response time of questions vary , But what make question be answered faster than the other ? if questions in domains with small number of professionals ? is the length of question effects ? 
What make professionals late in the response ?!!!

In [ ]:
questions_Answers['number_words_inQuestion'] = questions_Answers['questions_body'].apply(lambda x: len(x.split(' ')))
questions_Answers.loc[(questions_Answers['respond_time'] <= 7), 'week'] = 1
questions_Answers.loc[(questions_Answers['respond_time'] > 7) & (questions_Answers['respond_time'] <= 14), 'week'] = 2
questions_Answers.loc[(questions_Answers['respond_time'] > 14) & (questions_Answers['respond_time'] <= 21), 'week'] = 3
questions_Answers.loc[(questions_Answers['respond_time'] > 21) & (questions_Answers['respond_time'] <= 28), 'week'] = 4
questions_Answers.loc[(questions_Answers['respond_time'] > 28), 'week'] = 5


ax = sns.countplot(x="week", data=questions_Answers)
ax.set_title('respond time duration in week')

print(questions_Answers.groupby('week').size())

what makes 3,140 questions took more than 1 month to answered ? 

is the lenght of question body significantly effect in the response time ? Let's see the correlation between response time and number of words in the question body !

In [ ]:
corr=questions_Answers.plot.scatter(x='number_words_inQuestion',y='respond_time')
corr.set_title('response time VS number_words_inQuestion')

seems there is no correlation at general view?!! let's also check differences between questions answered during first week and those which are answered after more than 1 month

In [ ]:
fast_response_words = questions_Answers[ questions_Answers['week']==1]['number_words_inQuestion']
slow_response_words = questions_Answers[ questions_Answers['week']==5]['number_words_inQuestion']

pal = sns.color_palette()

plt.figure(figsize=(18, 8))
plt.hist(fast_response_words, bins=40, range=[0, 80], color=pal[9], label='fast')
plt.hist(slow_response_words, bins=40, range=[0, 80], color=pal[3], alpha=0.4, label='slow')
plt.legend()
plt.xlabel('Number of words', fontsize=15)
plt.ylabel('Number of responses', fontsize=15)


seems that shorter questions are answered faster than the long ones ......
But this is by not 
seems it is not the question body length that affects these slow response !!
Are they belongs to industries with small number of proffesionals ? Let's see..

### Let's explore tags<a id='tag'></a> 

In [ ]:
tags.head(5)

In [ ]:
tags.nunique()

There are 16269 unique tags

### Let's Explore tags with tag_users<a id='tag_tgU'></a>

Which tags have the largest numbers of followers ?

In [ ]:
tag_users.head(5)

In [ ]:
tags_tagUsers = pd.merge(tags, tag_users, left_on='tags_tag_id',right_on='tag_users_tag_id', how='inner')
tags_tagUsers.head(5)

In [ ]:
all_tags_followed=tags_tagUsers['tags_tag_name'].str.cat(sep=' ')

wordcloud = WordCloud(width=1500, height=800).generate(all_tags_followed)

plt.figure(figsize=(20, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Telecommunications , Information technology and college are most followed hashtags

### Let's explore emails<a id='em'></a>

In [ ]:
emails.head(5)

In [ ]:
emails.isnull().any()

In [ ]:
emails_frequency=emails['emails_frequency_level'].value_counts()
print(emails_frequency)
ax = emails_frequency.plot.bar(x=emails_frequency.index, y=emails_frequency.values)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title('Email frequency level count')
ax.set_xlabel("Email freq.Level")

most subscribers have daily email notification

### Let's explore matches<a id='mat'></a>

In [ ]:
matches.head(5)

what is the top 10 mails containing questions ?

In [ ]:
top10_mails = matches.groupby('matches_email_id').size().sort_values(ascending=False).head(10)
ax = top10_mails.plot.bar(x=top10_mails.index, y=top10_mails.values)
ax.set_title('Top10 mails containing questions')

## Let's build the model

### First split data into train and test 

In [ ]:
X_train, X_test= train_test_split(questions, test_size=0.05, random_state=42)
questions_Answers = pd.merge(X_train, answers, left_on='questions_id',right_on='answers_question_id', how='inner')
questions_Answers_test = pd.merge(X_test, answers, left_on='questions_id',right_on='answers_question_id', how='inner')
t = pd.merge(tags, tag_questions, left_on='tags_tag_id', right_on='tag_questions_tag_id')
t_test = pd.merge(t,questions_Answers_test,left_on='tag_questions_question_id',right_on='answers_question_id')

In [ ]:
k=questions_Answers_test['answers_author_id'].isin(questions_Answers['answers_author_id'])
f=questions_Answers_test[k]
f=f.reset_index()
UQ=f['questions_body'].unique()

### Let's begin with content based recommender<a id='cb'></a>

In [ ]:
def process_text(df, col):
    df[col] = df[col].str.replace('[^\w\s]','') # replacing punctuations
    df[col] = df[col].str.replace('-',' ') # replacing dashes
    df[col] = df[col].str.replace('\d+','') # replacing digits
    df[col] = df[col].str.lower().str.split() # convert all str to lowercase    
    df[col] = df[col].apply(lambda x: [item for item in x if item not in stop]) # remove stopwords    
    df[col] = df[col].apply(' '.join) # convert list to str
    return df

In [ ]:
def merging(df1, df2, left, right):
    return df1.merge(df2, how="inner", left_on=left, right_on=right)

In [ ]:
def combine_authors(df):
    c = df.groupby('questions_id')['answers_author_id'].apply(list)
    df_z = merging(df, pd.DataFrame(c), 'questions_id', 'questions_id')
    df_z.drop('answers_author_id_x', axis=1, inplace=True)
    df_z['answers_author_id_y'] = df_z['answers_author_id_y'].apply(', '.join)
    df_z.drop_duplicates(inplace=True)
    return df_z

In [ ]:
Ques_Ans_sub = questions_Answers[['questions_title', 'questions_body', 'answers_author_id', 'questions_id']].copy()

UniqueQues = combine_authors(Ques_Ans_sub)

Ques_Prof = UniqueQues[['questions_id', 'answers_author_id_y']].copy()

UniqueQues.drop('answers_author_id_y', axis=1, inplace=True)
UniquesQues = process_text(UniqueQues, "questions_title") 
UniqueQues = process_text(UniqueQues, "questions_body")

In [ ]:
tf = TfidfVectorizer(analyzer='word',
                         ngram_range=(1,2),
                         min_df=3,
                         max_df=0.9,
                         stop_words='english')
lst=UniqueQues['questions_body']
tfidf_matrix = tf.fit(lst)
tfidfTrans= tfidf_matrix.transform(lst)

In [ ]:
def CosSim(QuesBody):
    TargetQues= tfidf_matrix.transform([QuesBody])
    tfidfTrans.shape
    cosine_sim = linear_kernel(TargetQues, tfidfTrans)
    q_titles = UniqueQues['questions_title']
    q_ids = UniqueQues['questions_id']
    indices = pd.Series(UniqueQues.index, UniqueQues['questions_title'])
    return cosine_sim,indices,q_titles,q_ids

In [ ]:
def get_recommendations_idx(df):
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:21]
    q_indices = [i[0] for i in sim_scores]
    scores= [i[1] for i in sim_scores]
    df['scores']=scores
    return q_indices

def get_recommendations(df):
    return q_titles.iloc[get_recommendations_idx(df)]
    
def get_questions_id(df):
    ls=q_ids.iloc[get_recommendations_idx(df)] 
    df['Ques_ID']=ls.tolist()
    return df  

### Let's test content based recommender<a id='test'></a>

#### note : test is the input question body (asked question)
to input your test just comment this line : test=f['questions_body'][5]
and put your input question body 

**For ex :**
test="What is the best road to learn data science ?"

In [ ]:
df=pd.DataFrame()
test=f['questions_body'][5]
cosine_sim,indices,q_titles,q_ids=CosSim(test)
get_questions_id(df)

In [ ]:
y=[i for i, value in enumerate(questions_Answers_test['questions_body']) if value in test]
ExpectedProf=questions_Answers_test['answers_author_id'][y]
ExpectedProf=ExpectedProf.reset_index()
ExpectedProf

In [ ]:
z=[i for i, value in enumerate(questions_Answers['answers_author_id']) if value in ExpectedProf['answers_author_id'][0]]
m=questions_Answers.iloc[z]
temp5=m['answers_question_id'].isin(df['Ques_ID'])
RecomProfs5=m[temp5]
RecomProfs5

In [ ]:
ExpectedProf['answers_author_id'][0]

In [ ]:
temp=answers['answers_question_id'].isin(df['Ques_ID'])
RecomProfs=answers[temp]
temp2=comments_profs['professionals_id'].isin(RecomProfs['answers_author_id'])
ActiveProfs=comments_profs[temp2]
ActiveProfs=ActiveProfs[['professionals_id', 'comments_date_added' ]].copy()
ActiveProfs['comments_date_added'] =pd.to_datetime(ActiveProfs.comments_date_added)
ActiveProfs.sort_values(by=['comments_date_added'], inplace=True, ascending=False)
ActiveProfs.drop_duplicates('professionals_id',inplace=True)
ActiveProfs['Active']=ActiveProfs['comments_date_added']>'2018-01-01 00:00:00 UTC+0000'

In [ ]:
RecomProfsk=pd.merge(RecomProfs,ActiveProfs,how='left',left_on='answers_author_id',right_on='professionals_id')
RecomProfs2=RecomProfsk[['answers_question_id','answers_author_id','Active']][:]
RecomProfs2.nunique()

Discovering the last date of professionals's answers

In [ ]:
temp3=answers['answers_author_id'].isin(RecomProfs2['answers_author_id'].unique())
ActiveAns=answers[temp3]
ActiveAns['answers_date_added'] =pd.to_datetime(ActiveAns.answers_date_added)
ActiveAns.sort_values(by=['answers_date_added'], inplace=True, ascending=False)
ActiveAns.drop_duplicates('answers_author_id',inplace=True)
ActiveAns['Active']=ActiveAns['answers_date_added']>'2018-01-01 00:00:00'
ActiveAns=ActiveAns[['answers_author_id','answers_question_id','Active']][:]

In [ ]:
RecomProfsx=pd.merge(RecomProfs2,ActiveAns,how='left',left_on='answers_author_id',right_on='answers_author_id')
RecomProfsy=RecomProfsx[['answers_question_id_x','answers_author_id','Active_x','Active_y']][:]
RecomProfsy.fillna(False,inplace=True)
RecomProfsy['Active']=RecomProfsy['Active_x'] | RecomProfsy['Active_y']
RecomProfsy.drop(['Active_x','Active_y'], axis=1, inplace=True)
RecomProfsy.rename(columns={'answers_question_id_x' :'questions_id'},inplace=True)

RecomProfsy

Recommending professionals by giving weight=5 to the cosine similarity and 2 for the Active status

The id's of recommended professionals by content based is **ToCollaborative**

In [ ]:
Recommended_Prof=pd.merge(RecomProfsy,df,how='inner',left_on='questions_id',right_on='Ques_ID')
Recommended_Prof.drop_duplicates(inplace=True)
Recommended_Prof['FinalWeight']=(5*Recommended_Prof['scores'])+(2*Recommended_Prof['Active'])
Recommended_Prof2=Recommended_Prof.groupby(['answers_author_id'],as_index=False).max()
Recommended_Prof2.sort_values(by=['FinalWeight'], inplace=True, ascending=False)
ToCollaborative=Recommended_Prof2['answers_author_id'].values
ToCollaborative

### collaborative filtering<a id='cf'></a>

In [ ]:
def getProfessionalTages(t,id):
    indices1 = [i for i, value in enumerate(t['answers_author_id']) if value == id]
    p = t['tags_tag_name'][indices1]
    comm=np.unique(list(p))
    return comm

In [ ]:
#elbow method to chosse the best number of clusters
def elbow(df1):
    distortions = []
    k = 50
    K = []
    while k < 101:
        print(k)
        K.append(k)
        kmeanModel = KMeans(n_clusters=k).fit(df1)
        kmeanModel.fit(df1)
        distortions.append(sum(np.min(cdist(df1, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df1.shape[0])
        k += 2

    # # Plot the elbow
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()
    return

In [ ]:
#merge the data
t = pd.merge(tags,tag_questions, left_on='tags_tag_id', right_on='tag_questions_tag_id')
t = pd.merge(t,questions_Answers,left_on='tag_questions_question_id',right_on='answers_question_id')
tags=t['tags_tag_name']

#get features form the question using tfidf vectirization
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100, min_df=2, stop_words='english',use_idf=True)
vec = vectorizer.fit(tags)
x=vec.transform(tags)
df1=x.toarray()

In [ ]:
#cluster the data using k means
km = KMeans(n_clusters=62, init='k-means++', max_iter=100, n_init=1).fit(x)
labels = km.labels_.tolist()

#now predict the new professional
ids=ToCollaborative
most_common=[]
for id in range(len(ids)):
    out=getProfessionalTages(t,ids[id])
    out = vec.transform(out)
    out=out.toarray()
    pred=km.predict(out)
    indices = [i for i, value in enumerate(labels) if value in pred]
    #print(indices)
    p=t['answers_author_id'][indices]
    comm=Counter(p)
    n=10
    if len(comm)<10:
        n=len(comm)
    arr=[]
    comm=comm.most_common(n)
    for a,b in comm:
        arr.append(a)
    most_common.append(arr)

In [ ]:
most_common

In [ ]:
output=dict()
for i in range(len(most_common)):
    for j in range (len(most_common[i])):
        if most_common[i][j] in output:
            value=output.get(str(most_common[i][j]))+1
            key=str(most_common[i][j])
            output.update({key: value})
        else:
            output.setdefault(str(most_common[i][j]), 1)
sorted_output = sorted(output.items(), key=operator.itemgetter(1),reverse=True)
n=10
if len(comm) < 10:
    n = len(sorted_output)

the final recommended professionals' ids is **sorted_output**

#### final output<a id='final'></a>

In [ ]:
sorted_output[0:n]

### How to use the model<a id='use'></a>

1. write the asked question body in the variable [test](#test)  

2.  the final ids of recommended professionals are in the variable [sorted_output](#final)